In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from sklearn.preprocessing import StandardScaler
import time

In [ ]:
M = np.load('li_N10_k2_reducedm.npy')
y = np.load('syntheticEnergies.npy')



In [ ]:
# scaler = StandardScaler()
# M_scaled = scaler.fit_transform(M)

ctrue, _, _, _ = np.linalg.lstsq(M, y, rcond=None)

M_modeled = M[:,:100]

In [ ]:
print(f"Shape of design matrix M: {M.shape}")

nsims = 30
max_samples = 700

# Start with enough samples to ensure the matrix is likely full rank
nstart = 5

In [ ]:
def run_simulation(samples, overhead_time, probs=None, nstart=50):
    if probs is None:
        # Uniform probability for random sampling
        probs = np.ones_like(samples, dtype=float)

    assert probs.size == samples.size

    res = {'errs': [], 'MTMinv_norm':[], 'c_err': [], 'runtime':[]}
    T = samples[:nstart]

    start_time = time.perf_counter()
    for i in range(samples.size - nstart + 1):
        MTM = M_modeled[T, :]
        yT = y[T]
        
        probsi = np.sqrt(probs[:nstart+i] * (nstart+i))
        
        # Solve the least squares problem on the sampled subset
        chat = np.linalg.lstsq(MTM / probsi.reshape(-1, 1), yT / probsi, rcond=None)[0]
        
        # Calculate error against the full-dataset model's predictions
        res['errs'].append(np.linalg.norm(y - M_modeled @ chat))
        res['c_err'].append(np.linalg.norm(ctrue - chat))
        try: 
            Minv = np.linalg.pinv(MTM)
            res['MTMinv_norm'].append(norm(Minv,ord=2))

        except np.linalg.LinAlgError:
            res['MTMinv_norm'].append(np.nan)
        
        T = samples[:nstart+i+1]
        res['runtime'].append(time.perf_counter() - start_time + overhead_time)
    return res


In [ ]:
RESULTS_RAND = []
print("\nRunning Random Sampling Simulations...")
for j in range(nsims):
    start_time = time.perf_counter()
    rand_state = np.random.RandomState(42 + j)
    samples = np.arange(M_modeled.shape[0])
    rand_state.shuffle(samples)
    samples = samples[:max_samples]
    
    uniform_probs_on_samples = np.ones_like(samples, dtype=float) / M_modeled.shape[0]
    RESULTS_RAND.append(run_simulation(samples, time.perf_counter() - start_time, uniform_probs_on_samples, nstart=nstart))
    print(f"  Finished simulation {j+1}/{nsims}")


In [ ]:
import pickle


In [ ]:
# --- Leverage Score Sampling Simulations ---
start_time = time.perf_counter()
Q, _ = np.linalg.qr(M_modeled)
leverage_scores = np.linalg.norm(Q, axis=1)**2

# Normalize to get sampling probabilities
ls_probs = leverage_scores / leverage_scores.sum()

RESULTS_LS = []
overhead_time = time.perf_counter() - start_time
print("\nRunning Leverage Score Sampling Simulations...")

for j in range(nsims):
    rand_state = np.random.RandomState(42 + j)
    # Sample without replacement using the leverage score probabilities
    ls_samples = rand_state.choice(M_modeled.shape[0], max_samples, p=ls_probs, replace=True)
    ls_probs_on_samples = ls_probs[ls_samples]
    RESULTS_LS.append(run_simulation(ls_samples, overhead_time, ls_probs_on_samples, nstart=nstart))
    print(f"  Finished simulation {j+1}/{nsims}")


In [ ]:
from optimal_design_sub_mod import run_simulation_greedy_sub_mod
# use profiling to figure out why runtime is shooting up at the end
RESULTS_OD_SUB_MOD_A = [run_simulation_greedy_sub_mod(M_scaled, np.arange(100), ctrue, nstart, max_samples, "A")[0]]
RESULTS_OD_SUB_MOD_V = [run_simulation_greedy_sub_mod(M_scaled, np.arange(100), ctrue, nstart, max_samples, "V")[0]]

In [ ]:
max_to_plot = max_samples 
min_to_plot = 5
save = True

keys = ['errs', 'MTMinv_norm', 'c_err', 'runtime']
for key in keys:
    if key == 'errs':
        ylabel = r"Error, $\|\hat{f}_{\mathcal{T}} - f\|_2$" 
    # elif key == 'A_norm':
    #     ylabel = r"$\|A\|$"
    elif key == 'MTMinv_norm':
        ylabel = r"$\|M_{\mathcal{TM}}^\dagger\|$"
    elif key == 'c_err':
        ylabel = r"$\|c - \hat{c}\|$"
    elif key == 'runtime':
        ylabel = 'Time (s)'

    savename = f"{key.lower()}_cluster_expansion.png"
    fig, ax = plt.subplots()
    for results, name in zip([RESULTS_RAND, RESULTS_LS, RESULTS_OD_SUB_MOD_A, RESULTS_OD_SUB_MOD_V], ['Random', 'Lev. Score', 'OD Sub Mod A', 'OD Sub Mod V']):
        metric = np.array([res[key] for res in results])
        if key == 'errs':
            metric /= M_modeled.shape[0]
        mean = metric.mean(axis=0)
        # std = metric.std(axis=0)

        l = ax.loglog(np.arange(nstart, len(mean) + nstart), mean, label=name, linestyle='-')


        # ax.fill_between(np.arange(nstart, len(mean) + nstart), mean, mean + std, color=l[0].get_color(), alpha=0.5 )
        # for err in errs:
        #     ax.plot(np.arange(nstart, len(mean) + nstart), err, alpha=0.25, color=l[0].get_color())


    ax.legend(title='Sampling Method', fontsize=11)
    ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)

    ax.grid(True, which="both", ls="--", linewidth=0.5)
    plt.tight_layout()
    if save:
        plt.savefig(savename, dpi=250, format='png')

In [ ]:
plt.plot